In [1]:
def jaccard_similarity_sentences(sentence1, sentence2):
    # Mise en minuscules + découpage en mots
    set1 = set(sentence1.lower().split())
    set2 = set(sentence2.lower().split())

    # Intersection et union
    intersection = set1.intersection(set2)
    union = set1.union(set2)

    # Calcul Jaccard
    similarity = len(intersection) / len(union) if union else 0
    return similarity

# Tests
s1 = "je mange une banane"
s2 = "je mange une mangue"
s3 = "il conduit une voiture"

print("J(s1, s2) =", jaccard_similarity_sentences(s1, s2))
print("J(s1, s3) =", jaccard_similarity_sentences(s1, s3))


J(s1, s2) = 0.6
J(s1, s3) = 0.14285714285714285
